# Задание 1

Соберем датасет из данных

In [858]:
import re
import pandas as pd
import numpy as np

In [859]:
dataset = pd.DataFrame(columns=['text', 'lang'])

In [860]:
with open("2.tmp", "r") as file:
    file_content = file.read()
    file_content = re.sub('\n\x00+', '', file_content)
    paragraphs = file_content.split("\n")

    paragraphs = list(filter(lambda p: len(p) > 0, paragraphs))
    
    chinese_dataset = pd.DataFrame({'lang': ['ch'] * len(paragraphs),
                                    'text': paragraphs})
    
    dataset = pd.concat([dataset, chinese_dataset])

In [861]:
with open("3.tmp", "r") as file:
    file_content = file.read()
    file_content = re.sub('\x00', '', file_content)
    file_content = re.sub('\n\s+\n', '\n\n', file_content)
    file_content = re.sub('\n[^\n]', '', file_content)
    file_content = re.sub('\n\n', '\n', file_content)
    paragraphs = file_content.split("\n")
    
    paragraphs = list(filter(lambda p: len(p) > 0, paragraphs))
    
    english_dataset = pd.DataFrame({'lang': ['en'] * len(paragraphs),
                                    'text': paragraphs})
    
    dataset = pd.concat([dataset, english_dataset])

In [862]:
with open("4.tmp", "r") as file:
    file_content = file.read()
    file_content = re.sub('\n\x00+', '', file_content)
    paragraphs = file_content.split("\n")
    
    paragraphs = list(filter(lambda p: len(p) > 0, paragraphs))

    japanese_dataset = pd.DataFrame({'lang': ['ja'] * len(paragraphs),
                                    'text': paragraphs})
    
    dataset = pd.concat([dataset, japanese_dataset])

In [863]:
class Classifier:
    def __init__(self, langs):
        self.langs = list(map(lambda lang: {'name': lang[0], 'ord': int(lang[1])}, langs))
    
    def detect(self, text):    
        text_ord = list(filter(lambda s: ord(s) > 64, list(text)))
        text_ord = np.median(list(map(ord, text_ord)))
        lang_index = np.argmin(list(map(lambda lang: np.abs(lang['ord'] - text_ord), self.langs)))
        return self.langs[lang_index]['name']
    
    def classify(self, data):
        return list(map(self.detect, data))

In [864]:
classifier = Classifier([('en', 5000), ('ja', 15000), ('ch', 25000)])

In [865]:
y_predict = classifier.classify(dataset['text'].values)

In [866]:
"Accuracy score: " + str(sum(dataset['lang'].values == y_predict) / len(dataset))

'Accuracy score: 1.0'

Не имеет смысла запускать `KFold` поскольку все классифицируется верно

# Задание 2

### Class

In [867]:
import base64

In [868]:
class Base64:    
    def encode(to_encode):
        symbols = np.array(
            bytearray(map(ord, 
                          list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/")))
        )
        
        to_encode = bytearray(to_encode)
        
        added = 0
        while len(to_encode) % 3 != 0:
            to_encode.append(0)
            added += 1
        
        grouped = np.array(to_encode).reshape((int(len(to_encode) / 3), 3))
        
        encode_group = lambda group: [((group[0] & 252) >> 2),
                                      ((group[0] & 3)  << 4) + ((group[1] & 240) >> 4),
                                      ((group[1] & 15) << 2) + ((group[2] & 192) >> 6),
                                      ((group[2] & 63)) ]
        
        regrouped = np.array(list(map(encode_group, grouped)))
        encoded = bytearray(symbols[regrouped.flatten()])
        
        for i in range(1, added + 1):
            encoded[-i] = ord('=')
        
        return bytes(encoded)
        
    def decode(to_decode):
        symbols = bytearray(map(ord, 
                          list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/")))
        
        
        to_decode = bytearray(to_decode)
        added = 0
        while to_decode[-(added + 1)] == ord('='):
            added += 1
            to_decode[-added] = symbols[0]
        
        to_decode = list(map(symbols.index, to_decode))
        grouped = np.array(to_decode).reshape((int(len(to_decode) / 4), 4))
        
        decode_group = lambda group: [((group[0]       << 2) + ((group[1] & 48) >> 4)),
                                      ((group[1] & 15) << 4) + ((group[2] & 60) >> 2),
                                      ((group[2] & 3)  << 6) + ((group[3]))]
        
        regrouped = np.array(list(map(decode_group, grouped)))
        if added > 0:
            return bytes(list(regrouped.flatten()[:-added]))
        else:
            return bytes(list(regrouped.flatten()))

In [869]:
2**5 + 2**4 + 2**3 + 2**2

60

### Encode testing

In [870]:
for i in np.random.randint(len(dataset), size=5):
    b = bytes(dataset.iloc[i]['text'], 'utf-8')
    print("My base64:", Base64.encode(b), sep="\n", end="\n\n")
    print("base64:", base64.b64encode(b), sep="\n", end="\n\n")

My base64:
b'44Gj44Gm6KiA44Gj44Gf77ya44CM44GT44GTIOODm+ODg+ODk+ODg+ODiOOBruS4reOBruWuneefs+OBp+OBme+8gSAn77yD44OU44OU44Kk44Oz44Gv44GX44Gw44KJ44GP44GX44Gm55yg44KK44Gr6JC944Gh44CB5pyo44CF44Gu5LiL44Gu5YiI44KK6Iqd55Sf44Gr5oyB44Gh5LiK44GS44KJ44KM44Gm6YGL44Gw44KM44Gf44CC44Gd44GT44Gn5b2844Gv5p+U44KJ44GL44GE44OZ44OD44OJ44Gu5LiK44Gr572u44GL44KM44CB5aSc44Gu5q6L44KK44Gu6YOo5YiG44KS55yg44Gj44Gf44CC44K144Og44Gv5b28'

base64:
b'44Gj44Gm6KiA44Gj44Gf77ya44CM44GT44GTIOODm+ODg+ODk+ODg+ODiOOBruS4reOBruWuneefs+OBp+OBme+8gSAn77yD44OU44OU44Kk44Oz44Gv44GX44Gw44KJ44GP44GX44Gm55yg44KK44Gr6JC944Gh44CB5pyo44CF44Gu5LiL44Gu5YiI44KK6Iqd55Sf44Gr5oyB44Gh5LiK44GS44KJ44KM44Gm6YGL44Gw44KM44Gf44CC44Gd44GT44Gn5b2844Gv5p+U44KJ44GL44GE44OZ44OD44OJ44Gu5LiK44Gr572u44GL44KM44CB5aSc44Gu5q6L44KK44Gu6YOo5YiG44KS55yg44Gj44Gf44CC44K144Og44Gv5b28'

My base64:
b'44GR44KL44Gf44KB44Gr5pyo44CF44KS55m744Gj44Gf44CC44Gd44KM44Gv6Iul6ICF44KS6KaL44Gk44GR44KL44Gf44KB44Gr56m044Gr5b+N44Gz6L6844KT44Gg44CC44Gd44KM44Gv56qT44KS6YCa44Gj44Gm5Y+X44GR5Y

In [871]:
test = []
for i in range(len(dataset)):
    b = bytes(dataset.iloc[i]['text'], 'utf-8')
    test.append(Base64.encode(b) == base64.b64encode(b))

In [872]:
str(len(test) / sum(test) * 100) + "% - is correct"

'100.0% - is correct'

### Decode testing

In [873]:
for i in np.random.randint(len(dataset), size=5):
    b = bytes(dataset.iloc[i]['text'], 'utf-8')
    print("Decode:", Base64.decode(Base64.encode(b)).decode('utf-8'), sep="\n", end="\n\n")
    print("Original:", dataset.iloc[i]['text'], sep="\n", end="\n\n")

Decode:
他們盡可能快地跋涉，穿過草坪，穿過厚厚的老樹葉;所有關於他們的雨水滴流淌。他們沒有說話，但是一直在回想起來，從一邊到另一邊。 ＃＃半小時後，皮皮說：“我希望我們沒有向南轉，太過不了長長的木頭！這不是一個非常寬闊的腰帶 - 我應該在最廣泛的地方說出一英里以上 - 我們現在應該通過它了。“＃＃”我們開始走在曲折上是不好的，“弗羅多說。 “這不會改善事情。讓我們繼續保持原樣！我不知道我是否想進入公開場合。“＃他們還有幾英里的路程。然後，太陽再次閃閃發光，雨水減弱。現在已經過了中午，他們覺得是午餐的時候

Original:
他們盡可能快地跋涉，穿過草坪，穿過厚厚的老樹葉;所有關於他們的雨水滴流淌。他們沒有說話，但是一直在回想起來，從一邊到另一邊。 ＃＃半小時後，皮皮說：“我希望我們沒有向南轉，太過不了長長的木頭！這不是一個非常寬闊的腰帶 - 我應該在最廣泛的地方說出一英里以上 - 我們現在應該通過它了。“＃＃”我們開始走在曲折上是不好的，“弗羅多說。 “這不會改善事情。讓我們繼續保持原樣！我不知道我是否想進入公開場合。“＃他們還有幾英里的路程。然後，太陽再次閃閃發光，雨水減弱。現在已經過了中午，他們覺得是午餐的時候

Decode:
しょう。しかし、現時点では、この事があなたにどのように来たのかを知る必要があり、それが十分に物語にな

Original:
しょう。しかし、現時点では、この事があなたにどのように来たのかを知る必要があり、それが十分に物語にな

Decode:
ンスがあれば、いつか、戻ってくるだろう。あなたが来たら歓迎するよ。しかし、今私は考えがある。それはす

Original:
ンスがあれば、いつか、戻ってくるだろう。あなたが来たら歓迎するよ。しかし、今私は考えがある。それはす

Decode:
‘Mr. Frodo, sir!’ cried am quaking. ‘Don’t let him hurt me, sir! Don’t le him turn me into anything unnatural! My old dad ould take on so. I meant no harm, on my honour, sr!’

Original:
‘Mr. Frodo, sir!’ cried am quaking. ‘Don

In [874]:
test = []
for i in range(len(dataset)):
    b = bytes(dataset.iloc[i]['text'], 'utf-8')
    test.append(Base64.decode(Base64.encode(b)) == b)

In [875]:
str(len(test) / sum(test) * 100) + "% - is correct"

'100.0% - is correct'